In [1]:
#Import 
import numpy as np 
from scipy.linalg import norm 
from numpy.random import multivariate_normal, randn # Probability distributions on vectors
from scipy.linalg import toeplitz
from scipy.linalg import svdvals

In [2]:
from SGD_GD import stoch_grad
from L2regpb import L2regLS
from utils import generate_pb_parameters

In [21]:
d = int(50)
n = int(1e4)
idx = np.arange(d)
lbda = 1. / n ** (0.5)
#lbda = 0
kappa = 1e3

np.random.seed(1)

# Ground truth coefficients of the model
x_model_truth = (-1)**idx * np.exp(-idx / 10.)

A, y = generate_pb_parameters(x_model_truth, n, d, kappa,lbda, std=0, corr=0.7)
ylog = np.sign(y) # Taking the logarithm for binary classification
yone = np.copy(ylog)
yone[ylog==-1] = 0 

pbL2reg = L2regLS(A,y,lbda,'l2')
pblogreg = L2regLS(A,ylog,lbda,'logistic11')

print(pbL2reg.fun(x_model_truth))

0.027582025551758843


In [ ]:
x0 = np.zeros(d)
x,obj_val,_ = stoch_grad(x0=x0,problem=pbL2reg,xtarget=x_model_truth,stepchoice=0,step0=3e-3, n_iter=5000,nb=1,with_replace=False,verbose=True,verbose_ite=True)

Stochastic Gradient, batch size= 1 / 1000000
  iter   |   fval   |  normit 
       0 | 2.17e+02 | 2.35e+00
       1 | 2.16e+02 | 2.34e+00
       2 | 2.15e+02 | 2.34e+00
       3 | 2.11e+02 | 2.32e+00
       4 | 2.11e+02 | 2.32e+00
       5 | 2.11e+02 | 2.32e+00
       6 | 2.11e+02 | 2.32e+00
       7 | 2.11e+02 | 2.32e+00
       8 | 2.10e+02 | 2.32e+00
       9 | 2.10e+02 | 2.32e+00
      10 | 2.07e+02 | 2.31e+00
      11 | 2.06e+02 | 2.30e+00
      12 | 2.04e+02 | 2.29e+00
      13 | 2.04e+02 | 2.29e+00
      14 | 2.02e+02 | 2.28e+00
      15 | 2.02e+02 | 2.28e+00
      16 | 2.01e+02 | 2.27e+00
      17 | 2.01e+02 | 2.27e+00
      18 | 1.99e+02 | 2.26e+00
      19 | 1.98e+02 | 2.25e+00
      20 | 1.98e+02 | 2.25e+00
      21 | 1.96e+02 | 2.24e+00
      22 | 1.96e+02 | 2.24e+00
      23 | 1.95e+02 | 2.24e+00
      24 | 1.94e+02 | 2.23e+00
      25 | 1.92e+02 | 2.23e+00
      26 | 1.91e+02 | 2.22e+00
      27 | 1.85e+02 | 2.18e+00
      28 | 1.85e+02 | 2.18e+00
      29 | 1.84e+02 | 2.1

In [215]:

def S2GD(x0,problem,xtarget,h,m,nu,eps_tol,plus=False,alpha=2,n_iter=100,verbose=True): 
    """
        S2GD Semi Stochastic Gradient Descent.
        
        Inputs:
            x0: Initial vector
            problem
            xtarget: Target minimum (unknown in practice!)
            h : stepsize if < 0, we choose the best parameters m,h,n_iter
            m : max of # of Stochastic Gradient per epoch
            nu : lower bound on mu (the strongly convex cste of f)
            plus : Boolean if True -> Run SG2D+ 
            alpha : SG2D+ scaling parameter for # of Stochastic Gradient per epoch
            n_iter: Number of iterations, used as stopping criterion
            verbose: Boolean indicating whether information should be plot at every iteration (Default: False)
            
        Outputs:
            x_output: Final iterate of the method (or average if average=1)
            objvals: History of function values (Numpy array of length n_iter at most)
            normits: History of distances between iterates and optimum (Numpy array of length n_iter at most)
    """
    objvals = []
    normits = []
    L = problem.lipgrad()
    mu = problem.cvxval()
    kappa = problem.kappa_val
    n = problem.n
    x = x0.copy()
    nx = norm(x)


    #Compute best parameters
    if h < 0 : 
        delta = eps_tol ** (1/n_iter)
        h = 1 / ((4/delta) * (L - mu) + 2*L)
        if nu == mu: 
            m = (4*(kappa - 1)/delta + 2*kappa) * np.log(2/delta + (2*kappa - 1)/(kappa - 1))
            m = int(m) + 1
        elif nu == 0:
            m = 8*(kappa - 1) / delta**2 + 8*kappa/delta + (2*kappa**2) / (kappa-1)
            m = int(m) + 1
        #n_iter = int(np.log(1/eps_tol)) + 1 
        h = h / 10
        print("h=",h,"m=",m,"n_iter=",n_iter)

    #Compute probability and expectaiton for t
    t_val = np.arange(1,m+1) 
    t_probs = (1- nu*h)**(m-t_val)
    t_probs /= np.sum(t_probs)
    t_expectation = np.dot(t_val,t_probs)


    k=0
    
    # Current objective
    obj = problem.fun(x) 
    objvals.append(obj)

    # Current distance to the optimum
    nmin = norm(x-xtarget)
    normits.append(nmin)
    
    if verbose:
        # Display initial quantities of interest
        if plus: 
            print("S2GD+, n=",n,".  Number of inner loop at each iteration:",alpha*n)
            print("Running a single pass of Stochastic Gradient:")
            print(' | '.join([name.center(8) for name in ["iter", "fval", "normit"]]))
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % nmin).rjust(8)]))
        else :
            print("S2GD, n=",n,".  Exepcted number of inner loop at each iteration:",t_expectation," in [",(m+1)/2,",",m,"[")
            print(' | '.join([name.center(8) for name in ["iter", "fval", "normit"]]))
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % nmin).rjust(8)]))
    
    if plus :
        ik = np.random.choice(n,n,replace=False)
        for i in ik : 
            x = x - h * problem.grad_i(x,i)

        obj = problem.fun(x)
        nmin = norm(x-xtarget)
        nx = norm(x)

        if verbose : 
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % nmin).rjust(8)]))
            print("Running S2GD with t=", alpha*n)

    # Main loop
    while (k < n_iter and nx < 10**100):

        #Compute full gradient 
        g = problem.grad(x)

        #init
        y = np.copy(x)

        #Draw the number of sto grad 
        if plus : 
            t = alpha * n
        else : 
            t = np.random.choice(t_val,size=1,p=t_probs)[0]
            print("t=",t)

        #Inner Loop
        for _ in range(t):
            ind = np.random.choice(n,1,replace=True)
            y[:] = y - h * (g + problem.grad_i(y,ind) - problem.grad_i(x,ind))
        
        #Update
        x = y 
        
        nx = norm(x) 
        obj = problem.fun(x)
        nmin = norm(x-xtarget)
        
        k += 1

        objvals.append(obj)
        normits.append(nmin)
        if verbose:
            print(' | '.join([("%d" % k).rjust(8),("%.2e" % obj).rjust(8),("%.2e" % nmin).rjust(8)])) 

        if abs(problem.fun(x)-problem.fun(xtarget)) < eps_tol*abs(problem.fun(x0)-problem.fun(xtarget)):
            print(problem.fun(x)-problem.fun(xtarget))
            print(eps_tol*abs(problem.fun(x0)-problem.fun(xtarget)))
            print("Algorithm end because it reached precision.")
            break
    
    print(problem.fun(x)-problem.fun(xtarget))
    print(eps_tol*abs(problem.fun(x0)-problem.fun(xtarget)))
    # Outputs
    x_output = x.copy()
    
    return x_output, np.array(objvals), np.array(normits)

In [216]:
x0 = np.zeros(d)
S2GD(x0=x0,problem=pbL2reg,xtarget=x_model_truth,h=-1,m=10,nu=pbL2reg.cvxval(),eps_tol=1e-8)

h= 0.0014408529183230728 m= 10090 n_iter= 100
S2GD, n= 10000 .  Exepcted number of inner loop at each iteration: 5170.142163813495  in [ 5045.5 , 10090 [
  iter   |   fval   |  normit 
       0 | 2.33e+01 | 2.35e+00
t= 3576
       1 | 4.05e+00 | 1.49e+00
t= 6223
       2 | 5.67e-01 | 5.64e-01
t= 189
       3 | 1.15e-01 | 3.62e-01
t= 8623
       4 | 3.96e-02 | 8.78e-02
t= 9824
       5 | 3.01e-02 | 4.25e-02
t= 542
       6 | 2.80e-02 | 2.95e-02
t= 170
       7 | 2.76e-02 | 2.70e-02
t= 1019
       8 | 2.75e-02 | 2.60e-02
t= 8504
       9 | 2.75e-02 | 2.43e-02
t= 4740
      10 | 2.75e-02 | 2.40e-02
t= 9321
      11 | 2.75e-02 | 2.44e-02
t= 9426
      12 | 2.75e-02 | 2.53e-02
t= 7471
      13 | 2.75e-02 | 2.62e-02
t= 9738
      14 | 2.75e-02 | 2.75e-02
t= 7175
      15 | 2.75e-02 | 2.83e-02
t= 7303
      16 | 2.75e-02 | 2.91e-02
t= 1266
      17 | 2.75e-02 | 2.93e-02
t= 3662
      18 | 2.75e-02 | 2.96e-02
t= 8395
      19 | 2.75e-02 | 3.04e-02
t= 5143
      20 | 2.75e-02 | 3.08e-02
t= 8995

(array([ 0.98608306, -0.91580556,  0.8064419 , -0.75056241,  0.65959735,
        -0.61570994,  0.53924633, -0.50416692,  0.44178788, -0.41267087,
         0.36148259, -0.33777851,  0.29623025, -0.27658811,  0.24266575,
        -0.22626902,  0.19875314, -0.18527012,  0.16227299, -0.15235021,
         0.13284281, -0.12465489,  0.10879267, -0.10143547,  0.08935479,
        -0.08319012,  0.07310466, -0.06814552,  0.06010979, -0.05571352,
         0.0490377 , -0.04575681,  0.04017028, -0.03680415,  0.03355062,
        -0.02942257,  0.02838068, -0.02347334,  0.02379519, -0.01858695,
         0.01971101, -0.01467341,  0.01681084, -0.01182914,  0.01373472,
        -0.00993815,  0.01082259, -0.00825819,  0.00896378, -0.00667336]),
 array([23.27949153,  4.05271601,  0.56717629,  0.11502749,  0.03960641,
         0.03008881,  0.02795229,  0.02760233,  0.02754056,  0.02752587,
         0.02752326,  0.02752134,  0.02752023,  0.02751966,  0.02751916,
         0.02751891,  0.02751873,  0.02751871,  0

In [ ]:
np.random.choice(t_val,size=1,p=t_probs)

In [172]:
t_val = np.arange(1,10+1) 
t_probs = (1- 0.3)**(t_val)
t_probs /= np.sum(t_probs)
t_expectation = np.dot(t_val,t_probs)
np.random.choice(t_val,size=1,p=t_probs),t_probs,t_val

(array([8]),
 array([0.30872059, 0.21610441, 0.15127309, 0.10589116, 0.07412381,
        0.05188667, 0.03632067, 0.02542447, 0.01779713, 0.01245799]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]))